In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：AutoML训练表格二元分类模型，用于导出至云端

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_online_export_cloud.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_online_export_cloud.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_online_export_cloud.ipynb">
      在Google Cloud笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK来创建表格二元分类模型，并导出为一个云模型，使用Google Cloud AutoML模型。

数据集

本教程使用的数据集是[银行营销](https://pantheon.corp.google.com/storage/browser/_details/cloud-ml-tables-data/bank-marketing.csv)。这个数据集不需要任何特征工程。您在本教程中使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML表格二元分类模型，然后将模型导出为TensorFlow SavedModel格式。您也可以使用`gcloud`命令行工具或在Cloud Console上在线创建AutoML模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 将模型从`Model`资源导出到Cloud Storage。
- 在本地下载模型。
- 进行本地预测。

费用

本教程使用 Google Cloud 的计费组件:

* Vertex AI
* Cloud Storage

了解有关 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

### 设置您的本地开发环境

如果您使用的是Colab或Google Cloud笔记本，则您的环境已满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了详细的满足这些要求的说明。以下步骤提供了一组精简的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端 shell中运行 `pip3 install jupyter`。

5. 要启动Jupyter，请在终端 shell中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的 GA 版本 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的软件包之后，您需要重新启动笔记本内核，使其能够找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**以下步骤是必需的，无论您的笔记本环境如何。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您将获得 $300 的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您将需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 为本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 在以 `!` 为前缀的行上运行作为 shell 命令，并且以 `$` 为前缀的 Python 变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以改变 `REGION` 变量，该变量用于本笔记本的其余操作。以下是支持Vertex AI的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶进行Vertex AI训练。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在一个实时教程会话中，您可能正在使用一个共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个时间戳，并在本教程中创建的资源名称后附加时间戳。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud账户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照提示进行身份验证。

**否则**，请按照以下步骤进行：

在Cloud控制台中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入名称，然后点击**创建**。

在**为此服务账号授予对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入“Vertex”， 并选择**Vertex管理员**。在筛选框中输入“存储对象管理员”，并选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境。

将您的服务账号密钥的路径输入到下面的单元格中，将其作为GOOGLE_APPLICATION_CREDENTIALS变量运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论您的笔记本环境如何，都需要完成以下步骤。**

当您初始化用于Python的Vertex SDK时，您需要指定一个云存储暂存桶。暂存桶是在会话之间保留与数据集和模型资源相关的所有数据的位置。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化 Python 的 Vertex SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#教程

现在您已经准备好开始创建自己的AutoML表格二元分类模型。

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中 CSV 索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

### 快速查看您的数据

本教程使用存储在公共云存储桶中的一个版本的银行营销数据集，使用一个CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

您还需要知道用于训练的标签列的标题名称，该名称保存为`label_column`。对于这个数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `bq_source`：或者，从BigQuery表导入数据项到`Dataset`资源中。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="Bank Marketing" + "_" + TIMESTAMP, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：训练模型的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用到输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTabularTrainingJob(
    display_name="bank_" + TIMESTAMP,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

运行训练管道

接下来，通过调用方法`run`并传入以下参数来运行DAG以启动训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则在服务认为无法进一步改进模型目标测量时，训练可能会在使用整个预算前完成。

完成时，`run`方法将返回`Model`资源。

训练管道的执行将需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="bank_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 查看模型评估分数
在模型训练完成后，您可以查看其评估分数。

首先，您需要获取新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=bank_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

导出为云模型

您可以将AutoML表格二元分类模型导出为TensorFlow SavedFormat模型，然后可以自定义部署到云存储或本地下载。使用方法`export_model()`来将模型导出到云存储，该方法需要以下参数：

- `artifact_destination`：要将SavedFormat模型工件存储到的Cloud Storage位置。
- `export_format_id`：要将模型格式保存为的格式。对于AutoML表格二元分类，只有一个选项：
   - `tf-saved-model`：TensorFlow SavedFormat
- `sync`：是否同步或异步执行操作。

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_NAME, export_format_id="tflite", sync=True
)

model_package = response["artifactOutputUri"]

### 运行导出的模型包

您的模型现在以 TensorFlow SavedModel 格式存储，同时在 Cloud 存储桶中还附带一个模型服务器二进制文件。从 Cloud 存储桶中本地加载模型，然后您可以做一些事情，比如使用预构建的用于运行模型服务器的 Google Cloud 容器，使用 Docker 在本地部署模型和服务二进制文件。

#### 模型包布局

让我们开始查看导出包中的内容。导出的模型包将存储在您的 Cloud 存储桶中，如下所示：

    <your-gcs-bucket>/<model_name>/model-<model id>/tf-saved-model/<timedstamp>

接下来，看看模型包的内容。

In [ ]:
print("Model Package:", model_package)
print("Contents:")
! gsutil ls $model_package

print("\nTF Saved Model")
path = model_package + "/predict"
files = ! gsutil ls $path
saved_dir = files[1]
print(saved_dir)
! gsutil ls $saved_dir

#### 下载导出模型包。

您的第一步是将导出的模型包下载到本地。

In [ ]:
! gsutil cp -r $model_package .

将模型包目录名称重命名

在本教程中，您将使用Docker启动模型包。包路径名称中最后一个组件的时间戳包含一个“：”，这对Docker是无效的。您需要重命名。在下一步中，我们将最后一个组件重命名为“tbl_exported”。

In [ ]:
subdir = model_package.split("/")[-1]
print(subdir)
! mv $subdir tbl_exported

### 运行导出的模型包

该模型包依赖于一个预构建的谷歌云容器，专门用于提供导出的 AutoML 表格二元分类模型：

"gcr.io/cloud-aiplatform/automl_tables/prediction_server"

当前 Vertex 端点不支持该容器。但是，您可以在本地或另一个您管理的计算实例上使用该容器启动模型服务器。

在本教程中，您将使用 `docker` 启动它。Docker 容器将在此笔记本的后台运行，并使用 `-d` 选项将其指定为 `docker`。

In [ ]:
MODEL_SERVER = "gcr.io/cloud-aiplatform/automl_tables/prediction_server"
PORT = 8081

docker_id = ! docker run -d -v `pwd`/tbl_exported:/models/default -p 8081:8080 -it $MODEL_SERVER

检查健康状态

我们需要让一点时间来加载模型。让我们暂停10秒，然后进行健康检查。如果健康检查返回为空，表示准备就绪。

In [ ]:
import time

time.sleep(10)
! curl -X GET http://localhost:8081/health

验证Docker镜像的加载

您还可以使用`docker container ls`命令来验证Docker镜像的加载情况。我们将添加`--latest`选项以获得关于最新加载的（即该镜像）的信息。

In [ ]:
! docker container ls --latest

## 发出预测请求

现在使用您导出的模型进行预测。

### 预测请求格式

预测请求的格式是一个JSON对象，形式如下：

    { "instances": [ { "column_name_1": value, "column_name_2": value, … } , … ] }

将您的预测请求放在一个文本文件中，例如：

    test.json

In [ ]:
INSTANCES = {
    "instances": [
        {
            "Age": "58",
            "Job": "managment",
            "MaritalStatus": "married",
            "Education": "teritary",
            "Default": "no",
            "Balance": "2143",
            "Housing": "yes",
            "Loan": "no",
            "Contact": "unknown",
            "Day": "5",
            "Month": "may",
            "Duration": "261",
            "Campaign": "1",
            "PDays": "-1",
            "Previous": "0",
            "POutcome": "unknown",
        }
    ]
}

import json

with open("test.json", "w") as f:
    data = json.dumps(INSTANCES)
    f.write(data)

发送预测请求

您可以使用 CURL 发送预测请求：

    curl -X POST --data @test.json http://localhost:8081/predict

In [ ]:
! curl -X POST --data @/home/jupyter/test.json http://localhost:8081/predict

关闭Docker容器

In [ ]:
print(docker_id)
! docker kill $docker_id

# 清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME